In [46]:
import numpy as np
from dataclasses import dataclass
from matplotlib import pyplot as plt
from typing import Callable

In [19]:
# Memory dataclasses
@dataclass
class NeuronForMemory:
    def __init__(
            self,
            param_leak_str:     int,
            param_threshold:    int,
            param_reset:        int,
            state_core:         int
    ):
        self.leak_str   = param_leak_str
        self.threshold  = param_threshold
        self.reset      = param_reset
        self.core       = state_core
    def __repr__(self):
        return f"NeuronForMemory(leak_str={self.leak_str}, threshold={self.threshold}, reset={self.reset}, core={self.core})"

@dataclass
class SynapseForMemory:
    length: int
    word_size: np.dtype

    def __post_init__(self):
        self.weights = np.zeros(self.length, dtype=self.word_size)

    def __getitem__(self, key):
        return self.weights[key]
    
    def __setitem__(self, key, value):
        self.weights[key] = value

    def __len__(self):
        return self.length

    def __repr__(self):
        return f"SynapseForMemory(length={self.length}, weights={self.weights})"


In [23]:
# test dataclasses
print(NeuronForMemory(1, 2, 3, 4))
print(SynapseForMemory(10, np.int8))

NeuronForMemory(leak_str=1, threshold=2, reset=3, core=4)
SynapseForMemory(length=10, weights=[0 0 0 0 0 0 0 0 0 0])


In [ ]:
def get_discrete_random_sample(mu: float, sigma: float, min: int, max: int) -> int:
    sample = np.random.normal(loc=mu, scale=sigma)
    return np.clip(
        round(sample),
        min,
        max
    )

def random_neuron() -> NeuronForMemory:
    return NeuronForMemory(
        param_leak_str=get_discrete_random_sample(2**3, 2, 0, 2**5),
        param_threshold=get_discrete_random_sample(2**6, 2**3, 0, 2**11),
        state_core=0,
        param_reset=get_discrete_random_sample(4., 1., 0, 2**3)
    )

In [ ]:
@dataclass
class BRAM_Neuron:
    def __init__(
            self,
            length: int,
            randomize_fun: Callable[[], NeuronForMemory] = None
    ):
        self.length = length
        self.neurons = [NeuronForMemory(0, 0, 0, 0) for _ in range(length)]

        if randomize_fun:
            self.__randomize(randomize_fun)

    def __randomize(self, callback):
        for i in range(self.length):
            self.neurons[i] = callback()
        
    def __getitem__(self, key):
        return self.neurons[key]
    
    def __setitem__(self, key, value):
        self.neurons[key] = value

    def __len__(self):
        return len(self.neurons)
    
    def __repr__(self):
        return f"BRAM_Neuron(neurons={self.neurons})"
    

test_bram = BRAM_Neuron(48, random_neuron)
for i in range(48):
    print(test_bram[i])

NeuronForMemory(leak_str=9, threshold=74, reset=6, core=0)
NeuronForMemory(leak_str=6, threshold=65, reset=5, core=0)
NeuronForMemory(leak_str=5, threshold=57, reset=3, core=0)
NeuronForMemory(leak_str=7, threshold=69, reset=4, core=0)
NeuronForMemory(leak_str=7, threshold=70, reset=6, core=0)
NeuronForMemory(leak_str=7, threshold=64, reset=4, core=0)
NeuronForMemory(leak_str=10, threshold=79, reset=4, core=0)
NeuronForMemory(leak_str=10, threshold=57, reset=5, core=0)
NeuronForMemory(leak_str=11, threshold=72, reset=4, core=0)
NeuronForMemory(leak_str=9, threshold=69, reset=4, core=0)
NeuronForMemory(leak_str=8, threshold=65, reset=4, core=0)
NeuronForMemory(leak_str=9, threshold=71, reset=4, core=0)
NeuronForMemory(leak_str=11, threshold=53, reset=4, core=0)
NeuronForMemory(leak_str=9, threshold=70, reset=3, core=0)
NeuronForMemory(leak_str=10, threshold=62, reset=3, core=0)
NeuronForMemory(leak_str=7, threshold=71, reset=5, core=0)
NeuronForMemory(leak_str=9, threshold=63, reset=6, 

In [50]:
def random_synapse(depth: int) -> SynapseForMemory:
    
    temp_synapse = SynapseForMemory(depth, np.int8)
    for i in range(depth):
        temp_synapse[i] = get_discrete_random_sample(0.5, 1., 0, 3)
    return temp_synapse

In [51]:
@dataclass
class BRAM_synaptic_weights:
    def __init__(
            self,
            length: int,
            depth: int,
            word_size: np.dtype,
            randomize_fun: Callable[[int], SynapseForMemory] = None
    ):
        self.length = length
        self.depth = depth
        self.synapses = [SynapseForMemory(length, word_size) for _ in range(length)]

        if randomize_fun:
            self.__randomize(randomize_fun)
        
    def __randomize(self, callback):
        for i in range(self.length):
            self.synapses[i] = callback(self.depth)
    
    def __getitem__(self, key):
        return self.synapses[key]
    
    def __setitem__(self, key, value):
        self.synapses[key] = value

    def __len__(self):
        return (self.length, self.depth)
    
    def __repr__(self):
        return f"BRAM_synaptic_weights(synapses={self.synapses})"
    
test_bram_syn = BRAM_synaptic_weights(48, 48, np.int8, random_synapse)

for i in range(48):
    print(test_bram_syn[i])
        

SynapseForMemory(length=48, weights=[0 0 0 0 2 0 1 2 1 0 0 0 1 0 2 0 1 0 2 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 2 1
 1 0 2 1 1 0 1 2 0 2 0])
SynapseForMemory(length=48, weights=[1 0 1 0 1 2 2 0 0 2 2 2 1 0 1 1 1 0 2 0 1 0 2 0 1 1 1 0 0 1 2 2 1 1 2 2 1
 0 0 2 2 1 1 0 1 0 0 0])
SynapseForMemory(length=48, weights=[1 2 1 0 0 0 1 0 0 0 0 0 0 0 0 2 2 0 1 0 0 0 0 1 3 0 1 0 2 0 0 0 1 2 0 0 0
 1 0 0 1 0 0 1 0 0 1 1])
SynapseForMemory(length=48, weights=[0 1 1 0 1 0 0 2 1 1 2 1 1 2 0 0 1 1 1 0 0 2 1 0 0 1 0 0 0 1 1 0 1 0 2 1 0
 1 1 1 0 1 1 0 0 0 0 0])
SynapseForMemory(length=48, weights=[2 0 0 0 1 0 0 1 0 1 1 0 3 1 0 2 1 0 1 0 0 1 1 1 1 2 1 2 0 1 0 1 0 2 0 1 2
 0 1 0 1 1 1 1 1 0 0 1])
SynapseForMemory(length=48, weights=[0 1 0 1 1 2 1 0 0 1 0 0 1 1 0 1 0 2 0 2 0 1 0 2 1 0 1 2 0 1 1 0 0 0 2 1 2
 0 0 0 1 2 0 1 1 2 0 0])
SynapseForMemory(length=48, weights=[2 1 1 1 0 0 2 0 1 0 2 0 1 1 0 1 1 1 0 2 0 0 0 0 2 2 2 0 1 1 0 0 0 1 2 1 0
 0 0 0 0 1 2 1 2 0 0 0])
SynapseForMemory(length=48, weights=[0 0 1 0 0 2

In [ ]:
class LIF:

    neuron: NeuronForMemory
    weight: int

    def change_neuron(self, neuron: NeuronForMemory):
        self.neuron = neuron

    def change_weights(self, weight: int):
        self.weight = weight
    
    def forward(self, input):

        current_state = self.neuron.core
        current_state += input * self.weight

        if current_state > self.neuron.threshold:

    

    